In [1]:
import ibmJupyterNotebookStyles
ibmJupyterNotebookStyles.apply_ibm_styles()

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Grover Optimizer

## Introduction

*Grover Adaptive Search* (GAS) <sup>[1](#Gilliam2019)</sup> is a quantum search algorithm developed for solving combinatorial optimization problems, alongside variational quantum algorithms such as Variational Quantum Eigensolver (VQE) and Quantum Approximate Optimization Algorithm (QAOA). The algorithm iteratively applies Grover search <sup>[2](#Grover1996)</sup> to find the optimum value of an objective function, by using the best-known value from the previous run as a threshold. The adaptive oracle used in GAS recognizes all values above or below the current threshold (for max and min respectively), decreasing the size of the search space every iteration that the threshold is updated, until an optimum is found.

In this notebook we will explore each component of ```GroverOptimizer```, which utilizes the techniques described in GAS, by minimizing a Quadratic Unconstrained Binary Optimization (QUBO) problem, as described in <sup>[1](#Gilliam2019)</sup>.

## Grover Adaptive Search: Intuition

In <sup>[1](#Gilliam2019)</sup> the idea is to cast an optimization as a search problem by doing the following;
finding the minimum of a polynomial which is a weighted sum of monomials (meaning binary digits or keys), the weights being the values. There are two main registers for *keys* and *values* using the quantum dictionary framework <sup>[3](#GilliamVenci2019)</sup>.

To find the minimum, three things are needed

1. Efficiently encode the search space (which is value of the polynomial in superposition); through the above
two registers of i) keys (meaning which monomial contributes, which does not) and ii) values (that are the coefficients or weights
to add to those monomials who contribute) $\rightarrow$ Sum of the monomials by composing rotations

2. Search for a candidate by Grover’s algorithm $\rightarrow$ Through two’s complement always the left bit is the sign bit
$\rightarrow$ So just look at negatives by looking at that bit

3. In any optimization for a search problem you need to narrow down the search space
(hence its adaptive); take the values of the polynomial and shift by a threshold;
so that lesser and lesser values remain negative; at the end only the minimum will be
left as negative.

<br>
<div>
<img src="aux_files/Twos complement.png" width="400"/>
</div>
<br>

## Grover Adaptive Search - Details
Grover Search, the core element of GAS, needs three ingredients:

1. A state preparation operator $A$ to construct a superposition of all states in the search space.

2. An oracle operator $O$, that recognizes the states of interest and multiplies their amplitudes by $-1$.

3. The Grover diffusion operator $D$, that multiplies the amplitude of the $|0\rangle_n$ state by $-1$.

While implementations of GAS vary around the specific use case, the general framework still loosely follows the steps described below.

<br>
<div>
<img src="aux_files/GAS-Algorithm.png" width="400"/>
</div>
<br>

```GroverOptimizer``` uses ```QuadraticProgramToNegativeValueOracle``` to construct $A_y$ such that it prepares an $n$-qubit key register to represent the equal superposition of all $|x\rangle_n$ and an $m$-qubit value register to (approximately) represent the corresponding $|Q(x)-y\rangle_m$. Then, all states with $(Q(x) - y)$ negative should be flagged by $O_y$. Note that in the implementation discussed, the oracle operator is actually independent of $y$, but this is not a requirement. For clarity, we will refer to the oracle as $O$ when the oracle is independent of $y$.

Put formally, ```QuadraticProgramToNegativeValueOracle``` constructs an $A_y$ and $O$ such that:

<br>
<div>
<img src="aux_files/a-o-dfn.png" width="400"/>
</div>
<br>

where $|x\rangle$ is the binary encoding of the integer $x$. 

At each iteration in which the threshold $y$ is updated, we adapt $A_y$ such that the function values are shifted up or down (for minimum and maximum respectively) by $y$. For example, in the context of finding the minimum, as the value of $y$ decreases, the search space (negative values) also decreases, until only the minimum value remains. A concrete example will be explored in the next section.

## Use Cases - Portfolio Optimization

Suppose an investment universe consisting of n assets, denoted by $i = 1,\dots, n$, their corresponding expected returns 
$\mu ∈ \mathbb{R}^n$ and covariance matrix $Σ ∈ \mathbb{R}^{n×n}$. Furthermore, we consider a given risk factor $q ≥ 0$, which determines the considered risk appetite. The resulting objective function is 

\begin{equation} 
\min_{x \in \{0, 1\}^n} qx^T Σx − µ^Tx    \;\; \;\;\;\;\;\;               (1)
\end{equation}


In other words, we want to minimize the weighted variance minus the expected portfolio return. Setting $q = 0$ implies 
a risk neutral investor, while increasing q increases its risk averseness. In the presented form, portfolio optimization 
is a QUBO problem. We can extend it by imposing a budget constraint of the form 
\begin{eqnarray}
\sum_{i=1}^{N} x_i = B        \;\; \;\;\;\;\;\; \;\; \;\;\;\;\;  \;\; \;\;\;                      (2)
\end{eqnarray}

where $B \in {0, . . . , n}$  denotes the number of assets to be chosen. In general, equality constraints can be recast as penalty terms 
\begin{eqnarray}
λ \big(\sum_{i=1}^{N} x_i\big)^2 = B        \;\; \;\;\;\;\;\;  \;\; \;\;\;\;\;                (3)
\end{eqnarray}

and added to the objective (since we minimize), where $\lambda > 0$ is a large number to enforce the constraint to be satisfied. 
This results in a quadratic term that will again lead to a QUBO problem. However, with the methodology introduced in GAS, we can also model more complex constraints, e.g. a budget inequality constraint of the form
\begin{eqnarray}
\sum_{i=1}^{N} c_i x_i \leq B         \;\; \;\;\;\;\;\;  \;\; \;\;\;\;\;                 (4)
\end{eqnarray} 
where $c_i \in R$ denote the asset prices, which does usually make more sense in practice than (2). In the following we consider
two examples, one with an objection function that we want to minimize, and then the same problem with added constraints.

## Finding a Minimum Value
Consider a portfolio of n = 3 assets, risk factor q =
0.5, and returns described by:
$$
\mu =\begin{pmatrix}1\\ -2\\ 3\end{pmatrix} \  
\quad \text{and} \quad \Sigma =\begin{pmatrix}2&0&-4\\ 0&4&-2\\ -4&-2&10\end{pmatrix}  \;\; \;\;\;\;\;\;  \;\; (5)
$$
which leads to the formulation

\begin{eqnarray}
\min_{x_0,x_1,x_2 \in \{0, 1\}} -2x_0x_2 - x_1x_2 - x_0 + 2x_1 - 3x_2 \;\; \;\;\;\;\;\;  \;\; (6)
\end{eqnarray}


The objective function with added constant $−y$ (where $y$ is the current threshold) has an associated $A_y$ operator and the oracle $O$ recognizes negative values, as introduced in Sec. 3 of the paper. To perform the experiment we need 7 qubits split into two registers, $n = 3$ input qubits and 4 output qubits. While we only need 3 qubits in the output register, we add 1 extra to accommodate for the threshold shift. We set the initial threshold $y_1 = 0$, and stop searching if an improvement has not been found in three consecutive iterations of the algorithm.
For each iteration of GAS, we determine the number of applications of the Grover iterate as defined in Alg. 1 of the paper. We apply $A_y$ for the current threshold, and then apply the Grover iterate $G_y = A_yDA^\dagger_y O$ for the predetermined number of applications. If the measured value is less than y, we update the threshold.
This process repeats until we have seen no improvement for three consecutive iterations.
Classically, we can determine the original minimum
value by keeping track of the total shift, or by calculating the value of the objective function for the minimum key. The results of this simulated experiment
are shown in Figure below.

<br>
<div>
<img src="aux_files/Portfolio Optimization.png" width="800"/>
</div>
<br>

##  Additional Constraints

We can impose a budget inequality constraint of the form discussed in Eq. 4 to the previous problem, where $B < 2$ and the price of each asset is $1$. We can implement this constraint by adding an additional register to the existing quantum circuit, and then encode the Hamming weight of the binary representation of the keys in that register. Note that we only need to control on the most significant qubit of the constraint register to determine if $B < 2$. Otherwise, the procedure for applying GAS
is the same as in the original problem.

For additional insights on the portfolio problem the reader is encouraged to look at the GAS section of this notebook - https://github.com/stefan-woerner/uimp_csic_2020_summer_school_on_quantum_computing/blob/master/quantum_optimization_with_qiskit.ipynb

## Find the Minimum of a QUBO Problem using GroverOptimizer
The following is a toy example of a QUBO problem used in the above used case.

\begin{eqnarray}
\min_{x_0,x_1,x_2 \in \{0, 1\}} Q(x_0,x_1,x_2) = \min_{x_0,x_1,x_2 \in \{0, 1\}} -2x_0x_2 - x_1x_2 - x_0 + 2x_1 - 3x_2.
\end{eqnarray}

For our initial steps, we create a docplex model that defines the problem above, and then use the `from_docplex_mp()` function to convert the model to a `QuadraticProgram`, which can be used to represent a QUBO in Qiskit Optimization.

#### Note: Make sure to install qiskit_optimization by !pip install qiskit_optimization if you do not have it

In [3]:
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_optimization.algorithms import GroverOptimizer, MinimumEigenOptimizer
from qiskit_optimization.translators import from_docplex_mp
from docplex.mp.model import Model
from qiskit_aer import Aer

backend = Aer.get_backend('statevector_simulator')

In [4]:
model = Model()
x0 = model.binary_var(name='x0')
x1 = model.binary_var(name='x1')
x2 = model.binary_var(name='x2')
model.minimize(-x0+2*x1-3*x2-2*x0*x2-1*x1*x2)
qp = from_docplex_mp(model)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: - x0 + 2 x1 - 3 x2 + [ - 4 x0*x2 - 2 x1*x2 ]/2
Subject To

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1
 0 <= x2 <= 1

Binaries
 x0 x1 x2
End



Next as we use `GroverOptimizer` we need to specify the number of value qubits $m$ in the first argument `num_value_qubits` so that the value register can encode all possible values of $Q(x)-y$ in two's complement <sup>[4](#Wikipedia)</sup>. In order to specify an adequate number we can calculate a bound for the minimum with the sum of the absolute values of the coefficients of $Q(x)$. The magnitude of the bound is $|-1|+|+2|+|-3|+|-2|+|-1| = 1+2+3+2+1 = 9$ therefore the bound is $\pm 9$. To encode $\pm 9$ in two's complement we need $m=5$ qubits which has an integer range of $[-16, 15]$. For $n$ qubits the range is $\textstyle{[-2^{n-1}, 2^{n-1} -1]}$. This bound on $Q(x)$ further covers the range of $Q(x)-y$ as required.

Note that if the problem is not in QUBO form, it first needs to be converted to QUBO to calculate the bound and register size as above. This can be done with the Qiskit class ```QuadraticProgramToQubo```.

We instantiate a `GroverOptimizer` class with ```num_value_qubits=5```, and will terminate after there have been 10 iterations of GAS without progress (i.e. the value of $y$ does not change). The `solve()` function takes the `QuadraticProgram` we created earlier, and returns a results object that contains information about the run.

In [5]:
grover_optimizer = GroverOptimizer(num_value_qubits=5, num_iterations=10, quantum_instance=backend)
results = grover_optimizer.solve(qp)
print("x={}".format(results.x))
print("fval={}".format(results.fval))

x=[1. 0. 1.]
fval=-6.0


This results in the optimal solution $x_0=1$, $x_1=0$, $x_2=1$ and the optimal objective value of $-6$.
In the following, a custom visualization of the quantum state shows a possible run of `GroverOptimizer` applied to this QUBO.

<br>
<div>
<img src="aux_files/GAS-QUBO-Example.png" width="800"/>
</div>
<br>

Each graph shows a single iteration of GAS, with the current values of $r$ (iteration counter) and $y$ (threshold/offset) shown in the title. The X-axis displays the integer equivalent of the input (e.g. '101' $\rightarrow$ 5), and the Y-axis shows the possible function values. As there are 3 binary variables, there are $2^3=8$ possible solutions, which are shown in each graph. The color intensity indicates the probability of measuring a certain result (with bright intensity being the highest), while the actual color indicates the corresponding phase (see phase color-wheel below). Note that as $y$ decreases, we shift all of the values up by that amount, meaning there are fewer and fewer negative values in the distribution, until only one remains (the minimum).

<br>
<div>
<img src="aux_files/color-wheel-phase.png" width="200"/>
</div>

## Check that GroverOptimizer finds the correct value

We can verify that the algorithm is working correctly using the `MinimumEigenOptimizer` in Qiskit.

In [6]:
exact_solver = MinimumEigenOptimizer(NumPyMinimumEigensolver())
exact_result = exact_solver.solve(qp)
print("x={}".format(exact_result.x))
print("fval={}".format(exact_result.fval))

x=[1. 0. 1.]
fval=-6.0


### References

<a id='Gilliam2019'></a> 1. A. Gilliam, S. Woerner, C. Gonciulea. Grover Adaptive Search for Constrained Polynomial Binary Optimization; , [arXiv  preprint](https://arxiv.org/abs/1912.04088) arXiv:1912.04088, 2019 

<a id='Grover2019'></a> 2. Lov Grover. A fast quantum mechanical algorithm for database search; In Proceedings of the Twenty-eighth Annual ACM Symposium on Theory of Computing, STOC ’96, pages 212–219, New York, NY, USA, 1996. ACM. ISBN 0-89791-785-5. [Grover 1996](https://arxiv.org/abs/quant-ph/9605043)

<a id='GilliamVenci2019'></a> 3. Austin Gilliam, Charlene Venci, Sreraman Muralidharan, Vitaliy Dorum, Eric May, Rajesh Narasimhan, and Constantin Gonciulea. Foundational patterns for efficient quantum computing;  2019. [Gilliam, Venci 2019](https://arxiv.org/abs/1907.11513)

<a id='Wikipedia'></a> 4. Two's complement, [Wikipedia](https://en.wikipedia.org/wiki/Two%27s_complement)

<span style="font-size:10pt; font-weight:bold;"> &copy; Copyright IBM Corp. 2023 <br/>This content is categorized as an “existing” IBM asset as part of the IBM Quantum Accelerator contract. </span>